# Feature extraction example for RGB sideview images
The first step of image analysis is the segmentation, for this we recommend an instance segmentation algorithm. In the NPEC greenhouse a deeplab v3 algorithm is used to process the images.

After the plant is segmented from the image, information about the plant can be extracted like it's height, number of green pixels, width. Or additional deeplearning networks can be used to find fruits, flowers or other objects in the images. In this example will keep things simple and limit it the work to width, height and plant pixels.

In [12]:
# Imports
import os
import cv2
import colorsys
import math
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans

In [2]:
# Load data
foldername = "example_data"
mask_filename = foldername + os.sep + "mask.png"
mask_img = cv2.imread(str(mask_filename), -1)
mask_img = cv2.rotate(mask_img, cv2.ROTATE_90_CLOCKWISE)

masked_filename = foldername + os.sep + "masked.png"
masked_img = cv2.imread(str(masked_filename), -1)
masked_img = cv2.rotate(masked_img, cv2.ROTATE_90_CLOCKWISE)

Helper functions

In [ ]:
def boundingbox(mask_img, debug = False):
    """Determine width and height using a bounding box approach
    Draw a straigth boundingbox around the contour, as shown here:
    https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html
    """
    contours, _ = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if debug:
        debug_img = mask_img.copy()
        debug_img = cv2.cvtColor(debug_img, cv2.COLOR_GRAY2BGR) 
        # draw only contour 0, used for the height meassurements
        cv2.drawContours(debug_img, contours, 0, (255, 0, 0), 3)
        # Draw all contours
        # cv2.drawContours(debug_img, contours, -1, (255, 0, 0), 3)
        cv2.rectangle(debug_img, (x, y), (x + width, y + height), (0, 255, 0), 2)
        return debug_img, (x, y), width, height
    return (x, y), width, height

def ellipse_fit(mask_img, debug):
    """Fit an elipse to the first contour, find the minor and major axis
    Returns the axis closest to vertical as height and the other as width
    """
    contours, _ = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ellipse = cv2.fitEllipse(contours[0])
    width, height = compute_axis(ellipse)
    if debug:
        return ellipse, width, height
    return width, height


def compute_axis(ellipse, image=None):
    """Compute the start & end coordinates of the axis
    Called with the ellipse and optional the image to draw the axis on.
    see; https://stackoverflow.com/questions/62698756/opencv-calculating-orientation-angle-of-major-and-minor-axis-of-ellipse"""
    (xc, yc), (d1, d2), angle = ellipse
    radius = [d1/2, d2/2]
    radius.sort(reverse=True)

    # Longest axis red, shortest axis blue
    colors = [(255, 0, 0), (0, 0, 255)]
    angle_dist = {}
    for i, d_axis in enumerate(radius):
        if angle > 90:
            angle = angle - 90
        else:
            angle = angle + 90
        angle_dist[angle] = d_axis * 2
        x1 = xc + math.cos(math.radians(angle))*d_axis
        y1 = yc + math.sin(math.radians(angle))*d_axis
        x2 = xc + math.cos(math.radians(angle+180))*d_axis
        y2 = yc + math.sin(math.radians(angle+180))*d_axis
        if image is not None:
            cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), colors[i], 3)
    if image is not None:
        return image
    # Angle of 0 is horizontal, 90 degrees is vertical.
    width = angle_dist[min(angle_dist.keys())]
    height = angle_dist[max(angle_dist.keys())]
    return width, height

def resize_image(image, width=None, height=None, inter=cv2.INTER_AREA):
    ''' Function to resize an image and keep its original aspect ratio '''
    # initialize the dimensions of the image to be resized and grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the original image
    if width is None and height is None:
        return image
    if width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    return cv2.resize(image, dim, interpolation=inter)

def add_margin(param, img_shape, margin=50, axis=0):
    """Add margin to the bounding box coordinate for X or Y axis.
    axis=0 for Y (rows), axis=1 for X (columns)
    """
    size = img_shape[axis]
    param_min = max(param - margin, 0)
    param_max = min(param + margin, size)
    return param_min, param_max

## Shape analysis

In [4]:
def pixel_count(mask_img):
    """Return the number of non-zero pixels in the mask"""
    pixel_count = np.count_nonzero(mask_img != 0)
    return pixel_count

In [5]:
pixel_c = pixel_count(mask_img)
print(f"pixel count: {pixel_c}")

pixel count: 33351


In [35]:
debug_img, _, width, height = boundingbox(mask_img, True)
print(f"width: {width}, height: {height}")

ellipse, e_width, e_height = ellipse_fit(mask_img, True)
cv2.ellipse(debug_img, ellipse, (0, 255, 0), 3)
debug_img = compute_axis(ellipse, debug_img)
print(f"Ellipse width: {round(e_width,2)}, height: {round(e_height,2)}")

width: 216, height: 444
Ellipse width: 168.04, height: 398.48


In [33]:
# Example: Crop debug_img to bounding box with margin, resize, and display inline
margin = 50
(x, y), width, height = boundingbox(mask_img)
y_min, y_max = add_margin(y, debug_img.shape, margin, axis=0)
x_min, x_max = add_margin(x, debug_img.shape, margin, axis=1)
x_max = max(x_max, debug_img.shape[1])
cropped_img = debug_img[y_min:y+height+margin, x_min:x+width+margin]

if cropped_img.shape[0] == 0 or cropped_img.shape[1] == 0:
    print("Warning: Cropped image has zero height or width. Check the mask and bounding box.")
else:
    cropped_img = resize_image(cropped_img, height=700)
    fig = px.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB), title='Debug Preview (Cropped with Margin)')
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

## Color analysis
Segmentation is never perfect which explains the blue clusters. The other clusters and the number of pixels of pixels/cluster might in some cases contain information about the plants health.

In [7]:
def color_histogram(image, mask, hsv = False, n_clusters = 51):
    """ compute the counts per color in the image, with an option to select the number of return values
        Bins are used to group the colors
        MiniBatchKMeans sampled the background to often, instead the image is resized and fed into normal kmeans
        """
    if hsv:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # use boundingbox to only analyse the part of the image that contain plant
    (x, y), width, height = boundingbox(mask)
    plant_img = image[y:y+height, x:x+width, :]
    # apply k-means using the specified number of clusters and
    # then create the quantized image based on the predictions
    # Mini batch is used instead of normal kmeans to lower the computation time
    plant_img = plant_img.reshape((plant_img.shape[0] * plant_img.shape[1], 3))
    kmean = KMeans(n_clusters = n_clusters)
    labels = kmean.fit_predict(plant_img)
    clusters = kmean.cluster_centers_.astype("uint8")
    # Remove background cluster
    return clusters[clusters != [0, 0, 0]].reshape(n_clusters-1, 3)

def sort_by_hue(clusters):
    # Prepare data for plotting, sorted by hue (rainbow order) with hover info

    rgb_norm = clusters / 255.0
    hsv = np.array([colorsys.rgb_to_hsv(*rgb) for rgb in rgb_norm])
    idx = np.argsort(hsv[:,0])
    return clusters[idx]

In [9]:
clusters = color_histogram(masked_img, mask_img)
hsv_clusters = color_histogram(masked_img, mask_img, True)

In [10]:
sorted_rgb_clusters = sort_by_hue(clusters)
sorted_hsv_clusters = sort_by_hue(hsv_clusters)

sorted_cluster_colors = [f'rgb({int(r)},{int(g)},{int(b)})' for r, g, b in sorted_rgb_clusters]
sorted_hsv_colors = [f'rgb({int(r)},{int(g)},{int(b)})' for r, g, b in sorted_hsv_clusters]

# Add hover text with RGB values
rgb_hovertext = [f'RGB: ({int(r)}, {int(g)}, {int(b)})' for r, g, b in sorted_rgb_clusters]
hsv_hovertext = [f'RGB: ({int(r)}, {int(g)}, {int(b)})' for r, g, b in sorted_hsv_clusters]

fig = make_subplots(rows=2, cols=1, subplot_titles=("RGB Clusters (Rainbow Order)", "HSV Clusters (Rainbow Order)"))

fig.add_trace(
    go.Bar(x=list(range(len(sorted_rgb_clusters))), y=[1]*len(sorted_rgb_clusters), marker_color=sorted_cluster_colors, showlegend=False, hovertext=rgb_hovertext, hoverinfo="text"),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=list(range(len(sorted_hsv_clusters))), y=[1]*len(sorted_hsv_clusters), marker_color=sorted_hsv_colors, showlegend=False, hovertext=hsv_hovertext, hoverinfo="text"),
    row=2, col=1
)

fig.update_layout(height=400, width=800, title_text="Cluster Color Histograms (Rainbow Order)")
fig.update_yaxes(visible=False)
fig.show()